![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 2: RAG Fundamentals and Implementation

## Workshop: Context Engineering Matters

---

## Learning Objectives

By the end of this module, you will:

1. **Understand** what RAG is and why it matters for context engineering
2. **Implement** semantic search with vector embeddings
3. **Apply** keyword search with BM25 for exact matches
4. **Build** hybrid search combining semantic and keyword approaches
5. **Use** progressive disclosure for efficient context retrieval
6. **Compare** different RAG architecture patterns and their trade-offs

---

## Prerequisites

- Completed Module 1: Introduction to Context Engineering
- Redis Stack running (with vector search capability)
- OpenAI API key configured
- Course data loaded in Redis

---

## What is RAG?

**Retrieval Augmented Generation (RAG)** is a retrieval pattern that enhances LLM responses by:

1. **Retrieving** relevant information using semantic or keyword search
2. **Augmenting** the prompt (context) with this retrieved information
3. **Generating** responses grounded in the retrieved data

**Key Characteristic:** RAG uses **query-based search** (semantic similarity, keyword matching, or hybrid) rather than direct lookup by ID.

### Why RAG Matters

| Problem | RAG Solution |
|---------|-------------|
| LLMs have knowledge cutoff dates | Retrieve current information |
| LLMs can hallucinate facts | Ground responses in real data |
| LLMs don't know your domain | Inject domain-specific knowledge |
| Context windows are limited | Retrieve only relevant information |



## Why RAG Matters for Context Engineering

### The Challenge: Static vs. Dynamic Knowledge

In Section 1, we used **hardcoded** course information in the system context:

```python
system_context = """You are a Redis University course advisor.

Available Courses:
- RU101: Introduction to Redis (Beginner, 4-6 hours)
- RU201: Redis for Python (Intermediate, 6-8 hours)
...
"""
```

**Problems with this approach:**
- **Doesn't scale** - Can't hardcode thousands of courses
- **Wastes tokens** - Includes irrelevant courses in every request
- **Hard to update** - Requires code changes to add/modify courses
- **No personalization** - Same courses shown to everyone

### The Solution: Retrieval Augmented Generation (RAG)

RAG solves these problems by **dynamically retrieving** only the most relevant information:

```
User Query: "I want to learn about vector search"
     ↓
Embed Query → [0.23, -0.45, 0.67, ...]  # Convert text query to vector
     ↓
Semantic Search: Find courses matching "vector search"
     ↓
Retrieved Data: "RU301 - Vector Similarity Search with Redis"
     ↓
Context Assembly: Combine system + user + retrieved context
     ↓
LLM Generation: Generate response using retrieved context
```


**Benefits:**
- **Scales beyond tokens** - Store millions of documents
- **Token efficient** - Only retrieve what's relevant
- **Easy to update** - Add/modify data without code changes
- **Personalized** - Different results for different queries

### RAG and the Four Context Types

Remember the four context types from Module 1? **RAG is a retrieval pattern that can be applied to multiple context types**, though in this module we focus on Retrieved Context:

| Context Type | Storage | Retrieval Method | Example | RAG Applicable? |
|--------------|---------|------------------|---------|-----------------|
| System Context | Hardcoded | Always included | AI role, instructions | ❌ No (static) |
| User Context | Database | User ID lookup **+ optional RAG** | Student profile, **long-term memories** | ✅ Yes (Module 4) |
| Conversation Context | Session store | Session ID lookup **+ optional RAG** | Chat history, **conversation search** | ✅ Yes (advanced) |
| **Retrieved Context** | **Vector DB** | **RAG (semantic/keyword search)** | **Relevant courses, documents** | ✅ **Yes (this module)** |

**In this module**, we focus on using RAG for **Retrieved Context** (Pillar #4) - searching over domain knowledge like courses and documents.

**In Module 4**, you'll see RAG applied to **User Context** (Pillar #2) - searching over long-term memories to find relevant user facts.

**Key Insight:** RAG is not limited to one pillar. It's a retrieval pattern based on semantic/keyword search that can be applied wherever you need to find relevant information from a large collection.

---

Let's build each component step by step.

---

## Part 1: Setup and Configuration

In [ ]:
# Standard library imports
import os
import sys
from pathlib import Path

# Add project root to path for imports
# Handle both running from notebooks/ directory and from project root
if Path.cwd().name == "notebooks":
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables from project root
from dotenv import load_dotenv
env_path = project_root / ".env"
load_dotenv(dotenv_path=env_path)

# Verify required environment variables
required_vars = ["OPENAI_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"""
⚠️  Missing required environment variables: {', '.join(missing_vars)}

Please create a .env file in the project root with:
OPENAI_API_KEY=your_openai_api_key

For Redis setup:
- Local: docker run -d -p 6379:6379 redis/redis-stack-server:latest
- Cloud: https://redis.com/try-free/
""")
    raise SystemExit(1)

print("✅ Environment variables loaded")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")

# Third-party imports
import numpy as np
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage

# RedisVL imports for vector and hybrid search
from redisvl.index import SearchIndex
from redisvl.query import VectorQuery, FilterQuery, CountQuery
from redisvl.query.filter import Text, Tag, Num

# Project imports
from redis_context_course.redis_config import redis_config
from redis_context_course.hierarchical_manager import HierarchicalCourseManager
from redis_context_course.hierarchical_context import HierarchicalContextAssembler

print("✅ Imports successful!")

✅ Environment variables loaded
   OPENAI_API_KEY: ✓ Set
✅ Imports successful!


In [2]:
# Initialize clients
openai_client = OpenAI()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Initialize Redis connection (redis_config.redis_client is a property)
redis_client = redis_config.redis_client

# Initialize HierarchicalCourseManager for two-tier retrieval
hierarchical_manager = HierarchicalCourseManager(redis_client=redis_client)
context_assembler = HierarchicalContextAssembler()

# Initialize RedisVL SearchIndex reference for course summaries
# Note: course_details are stored as plain Redis keys, not in a vector index
try:
    summary_index = SearchIndex.from_existing("course_summaries", redis_url=redis_config.redis_url)
    print("""✅ Redis connection successful!
✅ HierarchicalCourseManager initialized
✅ HierarchicalContextAssembler initialized
✅ RedisVL SearchIndex initialized (course_summaries)""")
except Exception as e:
    # Index doesn't exist yet - it'll be created when data is loaded
    summary_index = None
    print(f"""✅ Redis connection successful!
✅ HierarchicalCourseManager initialized
✅ HierarchicalContextAssembler initialized
⚠️  SearchIndex not yet created (will be created on data load)""")

✅ Redis connection successful!
✅ HierarchicalCourseManager initialized
✅ HierarchicalContextAssembler initialized
✅ RedisVL SearchIndex initialized (course_summaries)


### Load Hierarchical Course Data

Before we can search, we need to load the hierarchical course data into Redis.
This only needs to be done once - the data persists in Redis.

**Note**: If you need to reload the data (e.g., after schema changes), set `FORCE_RELOAD = True`.

In [16]:
import json
from datetime import datetime
from redis_context_course.hierarchical_models import CourseSummary, CourseDetails, HierarchicalCourse

# Set to True to force reload even if data exists
FORCE_RELOAD = False

# Check if data is already loaded using RedisVL SearchIndex
try:
    # Try to get existing index and check document count
    check_index = SearchIndex.from_existing("course_summaries", redis_url=redis_config.redis_url)
    existing_count = check_index.info()['num_docs']
    needs_load = existing_count < 50 or FORCE_RELOAD

    if not needs_load:
        print(f"✅ Hierarchical course data already loaded ({existing_count} courses)")
except Exception:
    # Index doesn't exist yet
    needs_load = True
    existing_count = 0

if needs_load:
    if FORCE_RELOAD and existing_count > 0:
        print("🔄 Force reload requested - clearing existing data...")
        # Drop summary index (this also removes the summary data)
        try:
            summary_idx = SearchIndex.from_existing("course_summaries", redis_url=redis_config.redis_url)
            summary_idx.delete(drop=True)  # drop=True removes the index and data
            print("   Dropped course_summaries index")
        except Exception as e:
            print(f"   Note: course_summaries index not found ({e})")

        # Delete course_details keys (they're stored as plain Redis keys, not in an index)
        try:
            detail_keys = redis_client.keys("course_details:*")
            if detail_keys:
                redis_client.delete(*detail_keys)
                print(f"   Deleted {len(detail_keys)} course_details keys")
        except Exception as e:
            print(f"   Note: Error deleting course_details keys ({e})")

    print("📥 Loading hierarchical course data into Redis...")

    # Load from JSON file
    data_path = project_root / "src/redis_context_course/data/hierarchical/hierarchical_courses.json"
    with open(data_path, 'r') as f:
        data = json.load(f)

    courses_data = data.get("courses", [])
    print(f"   Found {len(courses_data)} courses in file")

    loaded = 0
    for course_data in courses_data:
        try:
            # Reconstruct HierarchicalCourse
            summary = CourseSummary(**course_data["summary"])
            details = CourseDetails(**course_data["details"])

            course = HierarchicalCourse(
                id=course_data["id"],
                summary=summary,
                details=details,
                created_at=datetime.fromisoformat(course_data["created_at"]),
            )
            
            # Add to Redis using await (Jupyter has an event loop)
            success = await hierarchical_manager.add_course(course)
            if success:
                loaded += 1
                if loaded % 10 == 0:
                    print(f"   Loaded {loaded}/{len(courses_data)} courses...")

            
        except Exception as e:
            print(f"   ❌ Error loading course: {e}")

    print(f"✅ Loaded {loaded} courses into Redis")

    # Reinitialize SearchIndex reference after loading
    summary_index = SearchIndex.from_existing("course_summaries", redis_url=redis_config.redis_url)
    print("✅ SearchIndex reference updated")

# Ensure SearchIndex reference is initialized (whether data was just loaded or already existed)
if summary_index is None:
    try:
        summary_index = SearchIndex.from_existing("course_summaries", redis_url=redis_config.redis_url)
        print("✅ SearchIndex reference initialized")
    except Exception as e:
        print(f"⚠️  Warning: Could not initialize SearchIndex: {e}")
        print("   Some examples below may not work until data is loaded.")

✅ Hierarchical course data already loaded (52 courses)


In [21]:
data_path = project_root / "src/redis_context_course/data/hierarchical/hierarchical_courses.json"
with open(data_path, 'r') as f:
    data = json.load(f)

courses_data = data.get("courses", [])
courses_data[0]

{'id': '01KBRB1FEW43EJCZT6TJKJADEG',
 'summary': {'course_code': 'CS001',
  'title': 'Introduction to Programming with Python',
  'department': 'Computer Science',
  'credits': 3,
  'difficulty_level': 'beginner',
  'format': 'hybrid',
  'instructor': 'Allison Hill',
  'short_description': 'Learn programming fundamentals using Python for beginners.',
  'prerequisite_codes': [],
  'tags': ['programming', 'python', 'beginner', 'fundamentals', 'intro'],
  'embedding_text': 'CS001: Introduction to Programming with Python | Department: Computer Science | Level: beginner | Learn programming fundamentals using Python for beginners. | Topics: programming, python, beginner, fundamentals, intro'},
 'details': {'course_code': 'CS001',
  'title': 'Introduction to Programming with Python',
  'department': 'Computer Science',
  'credits': 3,
  'difficulty_level': 'beginner',
  'format': 'hybrid',
  'instructor': 'Allison Hill',
  'full_description': 'Learn programming fundamentals using Python for b

---

## Part 2: Understanding Vector Embeddings

Before building our RAG system, let's understand the core concept: **vector embeddings**.

### What Are Embeddings?

Embeddings convert text into numerical vectors that capture semantic meaning:

```
Text: "Introduction to Redis"
  ↓ (embedding model)
Vector: [0.23, -0.45, 0.67, ..., 0.12]  # 1536 dimensions for OpenAI
```

**Key insight:** Similar texts have similar vectors (measured by cosine similarity).

### Why Embeddings Enable Semantic Search

Traditional keyword search:
- Query: "machine learning courses"
- Matches: Only documents containing exact words "machine learning"
- Misses: "AI courses", "neural network classes", "deep learning programs"

Semantic search with embeddings:
- Query: "machine learning courses"
- Matches: All semantically similar content (AI, neural networks, deep learning, etc.)
- Works across synonyms, related concepts, and different phrasings

Let's see this in action:

In [37]:
# Create embeddings for sample texts
sample_texts = [
    "Introduction to machine learning and neural networks",
    "Deep learning fundamentals with Python",
    "Advanced calculus and differential equations",
    "Redis database administration and optimization",
]

# Generate embeddings

response = openai_client.embeddings.create(
    model="text-embedding-3-small",
    input=sample_texts
)

# Extract embedding vectors
embeddings_list = [item.embedding for item in response.data]

print(f"""Vector Embeddings Demo
{'=' * 60}
Number of texts: {len(sample_texts)}
Embedding dimensions: {len(embeddings_list[0])}

Sample embedding (first 10 values):
{embeddings_list[0][:10]}...""")

14:40:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Vector Embeddings Demo
Number of texts: 4
Embedding dimensions: 1536

Sample embedding (first 10 values):
[-0.030388696119189262, -0.013120103627443314, 0.001388284144923091, -0.032543495297431946, 0.03266432508826256, -0.003478891449049115, -0.0007627381710335612, 0.04945966973900795, -0.02436734549701214, 0.07273954153060913]...


### Cosine Similarity

Cosine similarity measures the angle between two vectors:
- **1.0** = Identical meaning
- **0.0** = Unrelated
- **-1.0** = Opposite meaning (rare in practice)

Let's compare our sample texts:

In [38]:
def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors."""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Calculate similarity matrix
n = len(sample_texts)
similarity_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if i == j:
            similarity_matrix[i][j] = 1.0
        else:
            similarity_matrix[i][j] = cosine_similarity(embeddings_list[i], embeddings_list[j])

# Create short labels for display
short_labels = [
    "ML & Neural Nets",
    "Deep Learning",
    "Calculus",
    "Redis Admin"
]

# Display as a formatted table
print("Semantic Similarity Matrix")
print("=" * 70)
print("\nTexts:")
for i, text in enumerate(sample_texts):
    print(f"  [{i+1}] {short_labels[i]}: {text}")

print("\n" + "-" * 70)
print("\nSimilarity Matrix (1.0 = identical, 0.0 = unrelated):\n")

# Header row
header = "                          " + "  ".join(f"[{i+1}]   " for i in range(n))
print(header)
print("-" * 70)

# Matrix rows
for i in range(n):

    row = f"[{i+1}] {short_labels[i]:20s}"
    for j in range(n):
        row += f"  {similarity_matrix[i][j]:.3f} "
    print(row)

print("\n" + "=" * 70)
print("\nKey observations:")
print("  • ML & Neural Nets ↔ Deep Learning: HIGH similarity (0.525) - related topics")
print("  • ML topics ↔ Calculus: MODERATE similarity (0.27-0.31) - some overlap")
print("  • Redis Admin ↔ All others: LOW similarity (0.10-0.15) - different domain")

Semantic Similarity Matrix

Texts:
  [1] ML & Neural Nets: Introduction to machine learning and neural networks
  [2] Deep Learning: Deep learning fundamentals with Python
  [3] Calculus: Advanced calculus and differential equations
  [4] Redis Admin: Redis database administration and optimization

----------------------------------------------------------------------

Similarity Matrix (1.0 = identical, 0.0 = unrelated):

                          [1]     [2]     [3]     [4]   
----------------------------------------------------------------------
[1] ML & Neural Nets      1.000   0.525   0.305   0.095 
[2] Deep Learning         0.525   1.000   0.268   0.151 
[3] Calculus              0.305   0.268   1.000   0.142 
[4] Redis Admin           0.095   0.151   0.142   1.000 


Key observations:
  • ML & Neural Nets ↔ Deep Learning: HIGH similarity (0.525) - related topics
  • ML topics ↔ Calculus: MODERATE similarity (0.27-0.31) - some overlap
  • Redis Admin ↔ All others: LOW similarity 

### Key Insight: Semantic vs. Keyword Search

Notice how:
- "machine learning" and "deep learning" have HIGH similarity (related topics)
- "machine learning" and "calculus" have LOWER similarity (different domains)
- "Redis database" is distinct from all ML topics

This is the power of semantic search - it understands **meaning**, not just keywords.

---

## Part 3: Keyword Search with BM25

### The Limitation of Semantic Search Alone

While semantic search is powerful for understanding meaning, it has limitations:

```
Query: "Show me CS101"
Semantic Search: Might return CS102, CS201, or other CS courses
Problem: We want EXACT match on "CS101"

Query: "courses with prerequisites MATH020"
Semantic Search: Returns courses about math, prerequisites, etc.
Problem: We want courses that specifically require MATH020
```

**When Keyword Search Excels:**
- Exact course codes: "CS101", "MATH020", "BIO030"
- Specific terms: "prerequisites", "syllabus", "assignments"
- Names: "Dr. Smith", "Professor Johnson"
- IDs and codes: Error codes, product SKUs, ticket numbers

### BM25: Best Matching Algorithm

**BM25 (Best Matching 25)** is a ranking function used for keyword-based search:

- **TF (Term Frequency)**: How often does the term appear in the document?
- **IDF (Inverse Document Frequency)**: How rare is the term across all documents?
- **Document Length Normalization**: Adjusts for document length

**Formula intuition:**
```
BM25 Score = (term frequency × IDF) / (term frequency + k₁ × (1 - b + b × doc_length/avg_doc_length))

Where:
- k₁: Controls term frequency saturation (typical: 1.2)
- b: Controls length normalization (typical: 0.75)
```

### Redis Full-Text Search

Redis provides built-in full-text search capabilities with BM25 scoring:


In [22]:
# Demonstrate BM25 Keyword Search with RedisVL

print(f"""BM25 Keyword Search Demo
{'=' * 60}

📋 Searchable Fields in course_summaries Index:
   • Text fields (BM25): title, embedding_text
   • Tag fields (exact): course_code, department, difficulty_level
   • Vector field: embedding (for semantic search)
""")



# Example 1: Exact course code match using Tag filter
print("\n1️⃣ Exact Course Code Match (Tag Filter)")
print("   Query: 'CS001' (exact match)")

exact_match_query = FilterQuery(
    filter_expression=Tag("course_code") == "CS001",
    return_fields=["course_code", "title", "short_description", "difficulty_level"]
)
exact_results = summary_index.query(exact_match_query)

if exact_results:
    result = exact_results[0]
    print(f"   ✓ Found: {result['course_code']} - {result['title']}")
    print(f"     Difficulty: {result.get('difficulty_level', 'N/A')}")
else:
    print("   No exact match found")


# Example 2: Text search with BM25 scoring
print("\n2️⃣ Text Search with BM25 (Full-Text Search)")
print("   Query: 'programming' (BM25 scoring on title and description)")

# Search across multiple text fields for better recall
text_search_query = FilterQuery(
    filter_expression=(
        (Text("title") % "programming") |
        (Text("embedding_text") % "programming")
    ),
    return_fields=["course_code", "title", "short_description"]
)
text_results = summary_index.query(text_search_query)

print(f"   Found {len(text_results)} courses with 'programming' in title or description:")
for i, course in enumerate(text_results[:5], 1):
    print(f"   {i}. {course['course_code']}: {course['title']}")

# Example 3: Department-based search
print("\n3️⃣ Department Filter (Tag-based filtering)")
print("   Query: All Computer Science courses")

# Use department tag for broader matching
dept_query = FilterQuery(
    filter_expression=Tag("department") == "Computer Science",
    return_fields=["course_code", "title", "department"]
)
dept_results = summary_index.query(dept_query)

print(f"   Found {len(dept_results)} Computer Science courses:")
for i, course in enumerate(dept_results[:5], 1):
    print(f"   {i}. {course['course_code']}: {course['title']}")



BM25 Keyword Search Demo

📋 Searchable Fields in course_summaries Index:
   • Text fields (BM25): title, embedding_text
   • Tag fields (exact): course_code, department, difficulty_level
   • Vector field: embedding (for semantic search)


1️⃣ Exact Course Code Match (Tag Filter)
   Query: 'CS001' (exact match)
   ✓ Found: CS001 - Introduction to Programming with Python
     Difficulty: beginner

2️⃣ Text Search with BM25 (Full-Text Search)
   Query: 'programming' (BM25 scoring on title and description)
   Found 3 courses with 'programming' in title or description:
   1. CS001: Introduction to Programming with Python
   2. CS003: Programming Fundamentals with C++
   3. CS002: Web Development Fundamentals

3️⃣ Department Filter (Tag-based filtering)
   Query: All Computer Science courses
   Found 10 Computer Science courses:
   1. CS010: Cybersecurity Fundamentals
   2. CS005: Database Systems
   3. CS009: Computer Networks
   4. CS008: Operating Systems
   5. CS011: Cloud Computing


Key Insights:

- Tag filters: Exact matches (course codes, departments)
- Text filters: BM25 scoring for relevance ranking
- Wildcard/pattern matching: Find groups of related items
- Perfect for codes, IDs, and specific terms

### When to Use Keyword vs. Semantic Search

| Scenario | Best Approach | Why |
|----------|---------------|-----|
| "Show me CS101" | **Keyword** | Exact code match |
| "I want to learn AI" | **Semantic** | Understands intent, finds ML/DL/NLP |
| "courses requiring MATH020" | **Keyword** | Exact prerequisite match |
| "beginner programming courses" | **Semantic** | Understands "beginner" concept |
| "Dr. Smith's courses" | **Keyword** | Exact name match |
| "courses about databases" | **Semantic** | Finds Redis, SQL, NoSQL, etc. |

**The Problem:** In production, you often need BOTH!


---

## Part 4: Hybrid Search - Best of Both Worlds

### The Power of Combining Approaches

**Hybrid Search** combines semantic and keyword search to handle diverse query types:

```
User Query: "beginner CS courses about Python"
     ↓
┌─────────────────────────────────────────────────────────┐
│  SEMANTIC SEARCH              KEYWORD SEARCH             │
│  "beginner Python courses"    "CS" in course_code       │
│  ↓                            ↓                          │
│  [CS101, CS102, CS201,        [CS101, CS102, CS103,     │
│   MATH101, BIO101]            CS201, CS301]             │
└─────────────────────────────────────────────────────────┘
     ↓                            ↓
     └────────────┬───────────────┘
                  ↓
         Combined Scoring Algorithm
                  ↓
     [CS101, CS102, CS201, CS103]  ← Best results!
```



Hybrid search combines two complementary approaches:
1. **Semantic Search (Vector)**: Understands meaning and context
2. **Keyword Search (BM25)**: Finds exact words and phrases

**Why combine them?**
- Semantic search might miss exact terms (e.g., course codes like "CS101")
- Keyword search might miss conceptually similar content
- Together, they provide comprehensive results

**How does it work?**

There are two main approaches:

1. **Weighted Scoring**: Combine scores with a weight (alpha)
   - `hybrid_score = (alpha) * vector_score + (1-alpha) * text_score`
   - Example: alpha=0.7 means 70% vector, 30% text

2. **Reciprocal Rank Fusion (RRF)**: Combine rankings
   - `RRF_score(doc) = Σ (1 / (k + rank_i))` for each ranking
   - Documents appearing in multiple rankings get higher scores
   - More robust to score scale differences

RedisVL implements **weighted scoring** via `HybridQuery` (uses FT.AGGREGATE).


### Implementing Hybrid Search with RedisVL

Let's use RedisVL's native `HybridQuery` to combine semantic and keyword search:


In [7]:
# Import HybridQuery (it's actually AggregateHybridQuery)
from redisvl.query import HybridQuery

# Example: Hybrid search for "beginner programming courses"
query = "beginner programming courses"

print(f"""Hybrid Search Demo
{'=' * 60}
Query: "{query}"

🔍 How HybridQuery Works:
   1. Performs BM25 text search on 'embedding_text' field
   2. Performs vector search on 'embedding' field
   3. Combines scores: hybrid_score = (alpha)*vector + (1-alpha)*text
   4. Returns top N results sorted by hybrid score
""")

# Get query embedding for vector search
query_embedding = embeddings.embed_query(query)

# Create hybrid query with weighted scoring
hybrid_query = HybridQuery(
    text=query,                          # Text for BM25 search
    text_field_name="embedding_text",    # Text field to search
    vector=query_embedding,              # Vector for semantic search
    vector_field_name="embedding",       # Vector field to search
    text_scorer="BM25STD",               # BM25 scoring algorithm
    alpha=0.7,                           # Weight: 70% vector, 30% text
    num_results=10,
    return_fields=["course_code", "title", "short_description"],
    stopwords=None                       # No stopword filtering
)

# Execute hybrid search
hybrid_results = summary_index.query(hybrid_query)

print(f"\n📊 Hybrid Search Results (alpha=0.7):")
for i, course in enumerate(hybrid_results, 1):
    code = course.get('course_code', 'N/A')
    title = course.get('title', 'N/A')
    dept = code.split(str(i))[0] if code != 'N/A' and len(code) > 2 else code[:2]
    emoji = "💻" if dept.startswith("CS") else "📚"
    print(f"   {i}. {emoji} {code}: {title}")


Hybrid Search Demo
Query: "beginner programming courses"

🔍 How HybridQuery Works:
   1. Performs BM25 text search on 'embedding_text' field
   2. Performs vector search on 'embedding' field
   3. Combines scores: hybrid_score = (alpha)*vector + (1-alpha)*text
   4. Returns top N results sorted by hybrid score

12:59:10 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

📊 Hybrid Search Results (alpha=0.7):
   1. 💻 CS003: Programming Fundamentals with C++
   2. 💻 CS002: Web Development Fundamentals
   3. 📚 MATH023: Pre-Calculus
   4. 📚 BIO034: Introduction to Biology
   5. 📚 MATH031: Numerical Analysis
   6. 💻 CS010: Cybersecurity Fundamentals
   7. 💻 CS004: Data Structures and Algorithms
   8. 💻 CS018: Artificial Intelligence
   9. 📚 BIO042: Bioinformatics
   10. 📚 MATH025: Calculus II


/var/folders/5x/2sbds3f53fl1krk68wpww95h0000gp/T/ipykernel_44116/3245338558.py:22: DeprecationWarning: HybridQuery is a backward compatibility wrapper around AggregateHybridQuery and will eventually be replaced with a new hybrid query implementation. To maintain current functionality please use AggregateHybridQuery directly.
  hybrid_query = HybridQuery(



💡 Understanding the Results:

The alpha parameter (0.7) means:
  • 70% weight on semantic similarity (vector search)
  • 30% weight on keyword matching (BM25 text search)

Courses rank high when they:
  ✓ Are semantically similar to "beginner programming courses" (vector)
  ✓ Contain words like "beginner" or "programming" (text)
  ✓ Appear in BOTH rankings (get boosted by combined score)

⚠️  **Notice:** Some non-CS or programming courses appear (e.g., biology, cyber security)

**Why?** Because they match "beginner" and "introductory" semantically!

This is **CORRECT** hybrid search behavior - it finds semantically similar content even across different domains. The biology course IS a beginner/introductory course, so it's semantically relevant.

**To restrict to CS courses only**, add a filter - let's try it:


In [23]:
# Add department filter to restrict to Computer Science courses
from redisvl.query.filter import Tag

print(f"\n🎯 Hybrid Search with Department Filter")
print(f"{'=' * 60}")
print(f"Query: \"{query}\"")
print(f"Filter: department == 'Computer Science'\n")

# Create filtered hybrid query
filtered_hybrid_query = HybridQuery(
    text=query,
    text_field_name="embedding_text",
    vector=query_embedding,
    vector_field_name="embedding",
    text_scorer="BM25STD",
    alpha=0.7,
    num_results=10,
    return_fields=["course_code", "title", "department"],
    filter_expression=Tag("department") == "Computer Science",  # Add filter!
    stopwords=None
)

# Execute filtered search
filtered_results = summary_index.query(filtered_hybrid_query)

print(f"📊 Filtered Results (CS courses only):")
for i, course in enumerate(filtered_results, 1):
    code = course.get('course_code', 'N/A')
    title = course.get('title', 'N/A')
    print(f"   {i}. 💻 {code}: {title}")

print(f"""
{'=' * 60}
✅ Key Takeaway:

Hybrid Search (no filter):
  • Finds semantically similar courses across ALL departments
  • Good for exploratory search
  • May include unexpected but relevant results

Hybrid Search + Filter:
  • Combines semantic/keyword search WITH metadata filtering
  • Best for targeted search (e.g., "CS courses only")
  • Production-ready approach for most applications

💡 Filters can be combined:
   Tag("department") == "Computer Science" & Tag("difficulty_level") == "Beginner"
""")



🎯 Hybrid Search with Department Filter
Query: "beginner programming courses"
Filter: department == 'Computer Science'

📊 Filtered Results (CS courses only):
   1. 💻 CS003: Programming Fundamentals with C++
   2. 💻 CS001: Introduction to Programming with Python
   3. 💻 CS002: Web Development Fundamentals
   4. 💻 CS010: Cybersecurity Fundamentals
   5. 💻 CS018: Artificial Intelligence
   6. 💻 CS011: Cloud Computing
   7. 💻 CS019: Reinforcement Learning
   8. 💻 CS004: Data Structures and Algorithms
   9. 💻 CS007: Software Engineering
   10. 💻 CS014: Natural Language Processing

✅ Key Takeaway:

Hybrid Search (no filter):
  • Finds semantically similar courses across ALL departments
  • Good for exploratory search
  • May include unexpected but relevant results

Hybrid Search + Filter:
  • Combines semantic/keyword search WITH metadata filtering
  • Best for targeted search (e.g., "CS courses only")
  • Production-ready approach for most applications

💡 Filters can be combined:
   Tag("de

/var/folders/5x/2sbds3f53fl1krk68wpww95h0000gp/T/ipykernel_44116/606466081.py:10: DeprecationWarning: HybridQuery is a backward compatibility wrapper around AggregateHybridQuery and will eventually be replaced with a new hybrid query implementation. To maintain current functionality please use AggregateHybridQuery directly.
  filtered_hybrid_query = HybridQuery(


Try different alpha values:

  • alpha=1.0 → Pure semantic search (ignores keywords)

  • alpha=0.5 → Equal weight to semantic and keywords

  • alpha=0.0 → Pure keyword search (ignores semantics)

----

### Comparing Different Alpha Values

Let's see how different alpha values affect the results:


In [24]:
# Compare different alpha values
print(f"""Comparing Alpha Values
{'=' * 60}
Query: "{query}"
""")

for alpha_value in [1.0, 0.7, 0.5, 0.0]:
    hybrid_query = HybridQuery(
        text=query,
        text_field_name="embedding_text",
        vector=query_embedding,
        vector_field_name="embedding",
        text_scorer="BM25STD",
        alpha=alpha_value,
        num_results=5,
        return_fields=["course_code", "title"],
        filter_expression=Tag("department") == "Computer Science",  # Add filter!
        stopwords=None
    )

    results = summary_index.query(hybrid_query)

    # Determine search type description
    if alpha_value == 1.0:
        search_type = "Pure Semantic (100% vector)"
    elif alpha_value == 0.0:
        search_type = "Pure Keyword (100% BM25)"
    else:
        search_type = f"Hybrid ({int(alpha_value*100)}% vector, {int((1-alpha_value)*100)}% text)"

    print(f"\n🔍 Alpha = {alpha_value} ({search_type}):")
    for i, course in enumerate(results, 1):
        print(f"   {i}. {course.get('course_code', 'N/A')}: {course.get('title', 'N/A')}")



Comparing Alpha Values
Query: "beginner programming courses"


🔍 Alpha = 1.0 (Pure Semantic (100% vector)):
   1. CS010: Cybersecurity Fundamentals
   2. CS004: Data Structures and Algorithms
   3. CS018: Artificial Intelligence
   4. CS003: Programming Fundamentals with C++
   5. CS002: Web Development Fundamentals

🔍 Alpha = 0.7 (Hybrid (70% vector, 30% text)):
   1. CS003: Programming Fundamentals with C++
   2. CS002: Web Development Fundamentals
   3. CS010: Cybersecurity Fundamentals
   4. CS018: Artificial Intelligence
   5. CS004: Data Structures and Algorithms

🔍 Alpha = 0.5 (Hybrid (50% vector, 50% text)):
   1. CS003: Programming Fundamentals with C++
   2. CS002: Web Development Fundamentals
   3. CS010: Cybersecurity Fundamentals
   4. CS018: Artificial Intelligence
   5. CS004: Data Structures and Algorithms

🔍 Alpha = 0.0 (Pure Keyword (100% BM25)):
   1. CS003: Programming Fundamentals with C++
   2. CS002: Web Development Fundamentals
   3. CS010: Cybersecurity Fundame

/var/folders/5x/2sbds3f53fl1krk68wpww95h0000gp/T/ipykernel_44116/447379948.py:8: DeprecationWarning: HybridQuery is a backward compatibility wrapper around AggregateHybridQuery and will eventually be replaced with a new hybrid query implementation. To maintain current functionality please use AggregateHybridQuery directly.
  hybrid_query = HybridQuery(



📊 Key Observations:

• alpha=1.0 (Pure Semantic): Finds conceptually related courses
  - May include courses without "programming" or "beginner" in text
  - Good for exploratory search

• alpha=0.7 (Hybrid - Recommended): Balanced approach
  - Prioritizes semantic similarity but considers keywords
  - Best for most use cases

• alpha=0.5 (Equal Weight): Balanced hybrid
  - Equal importance to meaning and exact terms
  - Good when both aspects are equally important

• alpha=0.0 (Pure Keyword): Only exact text matches
  - Requires "programming" or "beginner" in the text
  - Good for precise term matching

🎯 Production Recommendation: Start with alpha=0.7 and tune based on user feedback


### Hybrid Search in Production

**Industry Best Practices** 

1. **Default to Hybrid**: Use hybrid search unless you have a specific reason not to
2. **Tune Weights**: Adjust semantic vs. keyword importance based on your domain
3. **Add Reranking**: Use a reranking model after hybrid search for best results
4. **Monitor Performance**: Track which search type contributes most to good results

**Advanced Hybrid Search Pipeline:**
```
Query → Hybrid Search (Semantic + Keyword) → Reranking Model → Top K Results → LLM
         ↓                                      ↓
    100 candidates                         10 best results
```

**Redis Stack Support:**
- ✅ Vector search (semantic)
- ✅ Full-text search (keyword/BM25)
- ✅ Combined queries (hybrid)
- ✅ Filtering and aggregations


---

## Part 4.5: When to Use Each Search Type

Now that you've learned all three search techniques, let's understand **when to use each one** in production systems.

### 🎯 Decision Guide

```
                        User Query
                            ↓
                    ┌───────┴───────┐
                    │ What type of  │
                    │ query is it?  │
                    └───────┬───────┘
                            ↓
        ┌───────────────────┼───────────────────┐
        ↓                   ↓                   ↓
    EXACT MATCH        CONCEPTUAL          COMPLEX
    (codes, IDs)       (natural lang)      (both)
        ↓                   ↓                   ↓
    Keyword/BM25       Vector Search       Hybrid Search
    (Tag/Text)         (Semantic)          (Combined)
```

### 📊 Search Type Comparison

| Search Type | Best For | Examples | When NOT to Use |
|-------------|----------|----------|-----------------|
| **Vector (Semantic)** | Natural language queries, conceptual similarity | "machine learning courses"<br>"What does the student prefer?"<br>"courses about databases" | Exact codes/IDs<br>Technical terms that must match exactly |
| **Keyword (BM25)** | Exact matches, codes, IDs, categories | "CS101"<br>"Computer Science" dept<br>"Python" (exact term) | Synonyms<br>Conceptual queries<br>Paraphrased questions |
| **Hybrid** | Complex queries with both semantic and exact requirements | "beginner CS programming courses"<br>"machine learning with Python"<br>"online data science classes" | Simple exact lookups<br>Pure conceptual queries |

### 🏗️ Real-World Examples from This Workshop

#### **1. Course Catalog Search** → Hybrid Search ✅

**Why?** Users ask both conceptual AND specific questions:
- "What machine learning courses are available?" (semantic)
- "Show me CS101" (exact match)
- "Beginner programming courses in Computer Science" (both!)

**Implementation:**
```python
# Option 1: HybridQuery (combines semantic + keyword)
hybrid_query = HybridQuery(
    text="beginner programming courses",
    text_field_name="embedding_text",
    vector=embedding,
    vector_field_name="embedding",
    alpha=0.7  # 70% semantic, 30% keyword
)

# Option 2: VectorQuery with filters (semantic + metadata)
filtered_query = VectorQuery(
    vector=embedding,
    filter_expression=Tag("difficulty_level") == "Beginner"
)
```

#### **2. Agent Memory Search** → Vector Search Only ✅

**Why?** Memory content is conversational, no exact codes:
- Memories: "Student prefers online courses", "Completed CS101 last semester"
- Queries: "What does the student prefer?", "What courses has the student taken?"
- No exact IDs to match, pure semantic understanding needed

**Implementation:**
```python
# Agent Memory Server uses vector search internally
results = await memory_client.search_long_term_memory(
    text="What does the student prefer?",
    user_id=UserId(eq=student_id)
)
```

#### **3. Course Details Storage** → No Index (Plain Keys) ✅

**Why?** Details are fetched by ID after summary search, never searched directly:
- Access pattern: Search summaries → Get IDs → Fetch details
- No need for vector embeddings (saves cost)
- Simple O(1) key-value lookup is perfect

**Implementation:**
```python
# Details stored as plain Redis keys
key = f"course_details:{course_id}"
details_json = redis_client.get(key)
```

### 🎓 Decision Framework

**Use Vector Search when:**
- ✅ Queries are natural language
- ✅ Need conceptual/semantic matching
- ✅ Synonyms and paraphrasing are common
- ✅ Dataset is conversational (like memories)

**Use Keyword Search when:**
- ✅ Need exact matches (codes, IDs, names)
- ✅ Technical terms must match precisely
- ✅ Filtering by categories/tags
- ✅ Fast exact lookups required

**Use Hybrid Search when:**
- ✅ Queries mix semantic and exact requirements
- ✅ Large, diverse dataset (like course catalogs)
- ✅ Production RAG systems (best practice)
- ✅ Need both recall (semantic) and precision (keyword)

**Use No Index (Plain Keys) when:**
- ✅ Data is fetched by ID only
- ✅ No search required
- ✅ Simple key-value access pattern
- ✅ Want to minimize cost and complexity

### 💡 Key Takeaway

**Different components need different search strategies!**

In this workshop:
- **Module 3 (Stages 1-3)**: Semantic search only (simple queries)
- **Module 4 (Memory)**: Semantic search only (conversational data)
- **Module 5 (Stage 4+)**: Hybrid search (production-ready)
- **Course Details**: No search (fetched by ID)

This is intentional and reflects real-world production patterns!


---

## Part 5: Progressive Disclosure with HierarchicalCourseManager

### The Problem: Information Overload

When building RAG systems, a common mistake is retrieving **too much context**:

```
User Query: "What machine learning courses are available?"

❌ Naive Approach: Load ALL course details (syllabi, assignments, prerequisites)
   → 50 courses × 500 tokens each = 25,000 tokens!
   → Costly and degrades quality

✅ Progressive Disclosure: Load summaries first, details on-demand
   → 50 summaries × 60 tokens = 3,000 tokens (for search)
   → Top 3 details × 500 tokens = 1,500 tokens (for response)
   → Total: 4,500 tokens (82% reduction!)
   → More efficient, better quality
```

### The Two-Tier Architecture

The `HierarchicalCourseManager` implements **progressive disclosure** with a two-tier data model:

```
┌─────────────────────────────────────────────────────────────────┐
│                    TIER 1: COURSE SUMMARIES                      │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐              │
│  │ CS101       │  │ ML201       │  │ DATA301     │  ...         │
│  │ ~60 tokens  │  │ ~60 tokens  │  │ ~60 tokens  │              │
│  │ Vector idx  │  │ Vector idx  │  │ Vector idx  │              │
│  └─────────────┘  └─────────────┘  └─────────────┘              │
│                         ↓ (semantic search)                      │
│                    Top K matches                                 │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│                    TIER 2: COURSE DETAILS                        │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │ ML201: Machine Learning Fundamentals                     │    │
│  │ ~500 tokens                                              │    │
│  │ • Full syllabus (12 weeks)                               │    │
│  │ • Prerequisites with descriptions                        │    │
│  │ • Assignments and projects                               │    │
│  │ • Instructor bio                                         │    │
│  └─────────────────────────────────────────────────────────┘    │
│  (fetched on-demand by course_code, metadata search)            │
└─────────────────────────────────────────────────────────────────┘
```

### Data Model Comparison

| Field | CourseSummary (~60 tokens) | CourseDetails (~500 tokens) |
|-------|---------------------------|----------------------------|
| course_code | ✅ | ✅ |
| title | ✅ | ✅ |
| department | ✅ | ✅ |
| difficulty_level | ✅ | ✅ |
| short_description | ✅ (100 chars) | ✅ |
| credits | ✅ | ✅ |
| format | ✅ | ✅ |
| full_description | ❌ | ✅ |
| syllabus | ❌ | ✅ (12 weeks) |
| prerequisites | ❌ | ✅ (with details) |
| assignments | ❌ | ✅ |
| instructor | ❌ | ✅ |
| instructor_bio | ❌ | ✅ |

### Why This Matters for LLMs

1. **"Lost in the Middle" Problem**: Research shows LLMs struggle with information in the middle of long contexts. Progressive disclosure keeps context focused.

2. **Token Economics**: Every input token adds cost. Reducing tokens saves $$$ per query.

3. **Response Quality**: Smaller, focused context might lead to better, more relevant responses.

4. **Latency**: Less data to process = faster responses.

### Key Methods

| Method | Purpose | Returns | Use Case |
|--------|---------|---------|----------|
| `search_summaries()` | Tier 1: Fast vector search | List[CourseSummary] | Initial search, browsing |
| `fetch_details()` | Tier 2: Get full course info | List[CourseDetails] | Deep dive on specific courses |
| `hierarchical_search()` | Combined two-stage retrieval | (summaries, details) | Complete search workflow |

### HierarchicalContextAssembler

The `HierarchicalContextAssembler` works with the manager to build optimized context:

```python
# Different assembly strategies
assembler.assemble_summary_only_context(query, limit=10)  # Lightweight overview
assembler.assemble_hierarchical_context(query, summary_limit=5, detail_limit=2)  # Progressive
assembler.assemble_with_budget(query, max_tokens=2000)  # Token-aware
```

### Demonstrate Tier 1: Search Summaries

Search course summaries using vector similarity.

In [25]:
# Demonstrate Tier 1: Search Summaries
query = "machine learning courses for beginners"

print(f"""Tier 1: Searching Course Summaries
{'=' * 60}
Query: "{query}"
""")

# Search summaries (lightweight)
summaries = await hierarchical_manager.search_summaries(
    query=query,
    limit=5
)

print(f"Found {len(summaries)} matching courses:\n")
for i, summary in enumerate(summaries, 1):

    print(f"""{i}. {summary.course_code}: {summary.title}
   Department: {summary.department} | Level: {summary.difficulty_level.value}
   {summary.short_description}
""")

Tier 1: Searching Course Summaries
Query: "machine learning courses for beginners"

13:52:51 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
13:52:51 redisvl.index.index INFO   Index already exists, not overwriting.
13:52:51 redis_context_course.hierarchical_manager INFO   Created summary index: course_summaries
13:52:51 redis_context_course.hierarchical_manager INFO   Found 5 course summaries for query: machine learning courses for beginners
Found 5 matching courses:

1. CS012: Machine Learning Fundamentals
   Department: Computer Science | Level: advanced
   Introduction to machine learning algorithms and applications.

2. MATH022: Linear Algebra for Machine Learning
   Department: Mathematics | Level: intermediate
   Vectors, matrices, and linear transformations for ML applications.

3. CS013: Deep Learning and Neural Networks
   Department: Computer Science | Level: advanced
   Advanced neural network architectures and deep learning techniques

### Demonstrate Tier 2: Fetch Details for Top Matches


In [26]:
# Demonstrate Tier 2: Fetch Details for Top Matches
top_course_codes = [s.course_code for s in summaries[:2]]

print(f"""Tier 2: Fetching Full Details
{'=' * 60}
Fetching details for: {top_course_codes}
""")

details = await hierarchical_manager.fetch_details(top_course_codes)

for detail in details:
    print(f"""
{'─' * 60}
{detail.course_code}: {detail.title}
Instructor: {detail.instructor}

Description:
{detail.full_description[:300]}...

Learning Objectives:""")
    for obj in detail.learning_objectives[:3]:
        print(f"  • {obj}")
    print(f"""
Syllabus: {detail.syllabus.total_weeks} weeks
Assignments: {detail.get_total_assignments()} total ({detail.get_total_points()} points)""")

Tier 2: Fetching Full Details
Fetching details for: ['CS012', 'MATH022']

13:53:07 redis_context_course.hierarchical_manager INFO   Fetched 2 course details

────────────────────────────────────────────────────────────
CS012: Machine Learning Fundamentals
Instructor: Jamie Arnold

Description:
Introduction to machine learning algorithms and applications. This course covers supervised and unsupervised learning, neural networks, and deep learning. Students will learn to implement ML algorithms from scratch and use popular frameworks like scikit-learn and TensorFlow....

Learning Objectives:
  • Understand core concepts in machine learning fundamentals
  • Implement machine learning fundamentals algorithms and techniques
  • Apply machine learning fundamentals to real-world problems

Syllabus: 15 weeks
Assignments: 10 total (1250 points)

────────────────────────────────────────────────────────────
MATH022: Linear Algebra for Machine Learning
Instructor: Patty Perez

Description:
Vectors,

### Demonstrate Combined Hierarchical Search

High-level workflow:
1. Search summaries (lightweight, fast)
2. Fetch details for top matches (comprehensive, on-demand)

In [30]:
# # Demonstrate Combined Hierarchical Search
# query = "advanced Redis data structures"

# print(f"""Hierarchical Search (Two-Stage Retrieval)
# {'=' * 60}
# Query: "{query}"
# """)

# summaries, details = await hierarchical_manager.hierarchical_search(
#     query=query,
#     summary_limit=5,  # Get 5 summaries
#     detail_limit=2    # But only 2 full details
# )

# print(f"""Stage 1: Retrieved {len(summaries)} summaries
# Stage 2: Fetched {len(details)} detailed courses

# All Matching Courses (Summaries):""")
# for i, s in enumerate(summaries, 1):
#     marker = "★" if i <= len(details) else "○"
#     print(f"  {marker} {s.course_code}: {s.title}")

# print(f"\n★ = Full details available | ○ = Summary only")

---

## Part 6: Building the Complete RAG Pipeline

Now let's combine everything into a complete RAG pipeline: Retrieval → Context Assembly → Generation.

### The RAG Flow

```
User Query
    ↓
1. Hierarchical Search (retrieve summaries → fetch details)
    ↓
2. Context Assembly (combine system + user + retrieved context)
    ↓
3. LLM Generation (create personalized response)
```

Let's implement each step:

In [31]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

print("✅ LLM initialized (gpt-4o-mini)")

✅ LLM initialized (gpt-4o-mini)


### Step 5.1: Retrieval Function with Hierarchical Search

First, let's create a function to retrieve relevant courses using hierarchical search:

In [32]:
async def retrieve_courses_hierarchical(
    query: str,
    summary_limit: int = 5,
    detail_limit: int = 3
):
    """
    Retrieve relevant courses using hierarchical search.

    Args:
        query: User's search query
        summary_limit: Number of summaries to retrieve
        detail_limit: Number of full details to fetch

    Returns:
        Tuple of (summaries, details)
    """
    summaries, details = await hierarchical_manager.hierarchical_search(
        query=query,
        summary_limit=summary_limit,
        detail_limit=detail_limit
    )
    return summaries, details


# Test retrieval
test_query = "I want to learn about data structures"
test_summaries, test_details = await retrieve_courses_hierarchical(test_query)

print(f"🔍 Retrieved for: '{test_query}'")
print(f"   Summaries: {len(test_summaries)}")
print(f"   Details: {len(test_details)}")
for s in test_summaries:
    print(f"   - {s.course_code}: {s.title}")

13:53:44 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I want to learn about data structures' (summaries=5, details=3)
13:53:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
13:53:45 redis_context_course.hierarchical_manager INFO   Found 5 course summaries for query: I want to learn about data structures
13:53:45 redis_context_course.hierarchical_manager INFO   Fetched 3 course details
13:53:45 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 5 summaries, 3 details
🔍 Retrieved for: 'I want to learn about data structures'
   Summaries: 5
   Details: 3
   - CS004: Data Structures and Algorithms
   - CS016: Data Science and Analytics
   - CS020: Big Data Technologies
   - CS003: Programming Fundamentals with C++
   - CS005: Database Systems


### Step 5.2: Context Assembly Function

Now let's assemble context from multiple sources (system + user + retrieved):

In [33]:
def assemble_rag_context(
    user_query: str,
    summaries: list,
    details: list,
    user_profile: dict = None
):
    """
    Assemble context from multiple sources for the LLM.

    This implements the context engineering principles from Module 1:
    - System Context: AI role and instructions
    - User Context: Student profile and preferences
    - Retrieved Context: Relevant courses from hierarchical search
    """

    # System Context: Define the AI's role
    system_context = """You are a Redis University course advisor.

Your role:
- Help students find courses that match their interests and goals
- Provide personalized recommendations based on student profiles
- Explain course prerequisites and learning paths
- Be encouraging and supportive

Guidelines:
- Only recommend courses from the provided course list
- Consider student's difficulty level preferences
- Explain your reasoning for recommendations
- Be concise but informative
"""

    # User Context: Student profile (if provided)
    user_context = ""
    if user_profile:
        user_context = f"""
Student Profile:
- Name: {user_profile.get('name', 'Student')}
- Major: {user_profile.get('major', 'Undeclared')}
- Year: {user_profile.get('year', 'N/A')}
- Interests: {', '.join(user_profile.get('interests', []))}
- Preferred Difficulty: {user_profile.get('preferred_difficulty', 'any')}
- Preferred Format: {user_profile.get('preferred_format', 'any')}
"""

    # Retrieved Context: Use HierarchicalContextAssembler
    retrieved_context = context_assembler.assemble_hierarchical_context(
        summaries=summaries,
        details=details,
        query=user_query
    )

    # Combine all context
    full_context = system_context
    if user_context:
        full_context += user_context
    full_context += "\n" + retrieved_context

    return full_context


# Test context assembly
test_profile = {
    "name": "Sarah Chen",
    "major": "Computer Science",
    "year": "Junior",
    "interests": ["machine learning", "data science"],
    "preferred_difficulty": "intermediate",
    "preferred_format": "online",
}

assembled_context = assemble_rag_context(
    user_query=test_query,
    summaries=test_summaries,
    details=test_details,
    user_profile=test_profile,
)

print("✅ Context assembled")
print(f"   Total length: {len(assembled_context)} characters")
print(f"   Includes: System + User + Retrieved context (hierarchical)")

✅ Context assembled
   Total length: 13722 characters
   Includes: System + User + Retrieved context (hierarchical)


In [34]:
print(f"Observe the assembled context: \n\n{assembled_context}")

Observe the assembled context: 

You are a Redis University course advisor.

Your role:
- Help students find courses that match their interests and goals
- Provide personalized recommendations based on student profiles
- Explain course prerequisites and learning paths
- Be encouraging and supportive

Guidelines:
- Only recommend courses from the provided course list
- Consider student's difficulty level preferences
- Explain your reasoning for recommendations
- Be concise but informative

Student Profile:
- Name: Sarah Chen
- Major: Computer Science
- Year: Junior
- Interests: machine learning, data science
- Preferred Difficulty: intermediate
- Preferred Format: online

# Course Search Results for: I want to learn about data structures

## Overview of All Matches

Found 5 relevant courses:


### 1. CS004: Data Structures and Algorithms
**Department**: Computer Science
**Instructor**: Daniel Wagner
**Credits**: 4 | **Level**: intermediate
**Format**: hybrid

Essential data structures a

**🎁 Bonus:** Can you identify the different parts of the context from what we learned in Module 1?

**✅ Answer:** Yes! Looking at the assembled context above, we can identify all three context types from Module 1:

1. **System Context** (Static)
   - The first section: "You are a Redis University course advisor..."
   - Defines the AI's role, responsibilities, and guidelines
   - Remains the same for all queries
   - Sets behavioral instructions and constraints

2. **User Context** (Dynamic, User-Specific)
   - The "Student Profile" section
   - Contains Sarah Chen's personal information: major, year, interests, preferences
   - Changes based on who is asking the question
   - Enables personalized recommendations

3. **Retrieved Context** (Dynamic, Query-Specific)
   - The "Relevant Courses" section with summaries and details
   - Uses progressive disclosure (summaries for all, details for top matches)
   - Changes based on the specific query
   - Provides the factual information the LLM needs to answer

Notice how all three work together: System Context tells the AI **how to behave**, User Context tells it **who it's helping**, and Retrieved Context provides **what information is relevant**. This is RAG in action!

### Step 5.3: Generation Function

Finally, let's generate a response using the assembled context:

In [35]:
async def generate_response(user_query: str, context: str):
    """
    Generate LLM response using assembled context.

    Args:
        user_query: User's question
        context: Assembled context (system + user + retrieved)

    Returns:
        LLM response string
    """
    messages = [SystemMessage(content=context), HumanMessage(content=user_query)]

    response = await llm.ainvoke(messages)
    return response.content

# test_query = "I want to learn about data structures"
# Test generation
response = await generate_response(test_query, assembled_context)

print("\n🤖 Generated Response:\n")
print(response)

13:54:49 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

🤖 Generated Response:

Hi Sarah! Based on your interest in data structures and your intermediate preferred difficulty level, I recommend the following course:

### CS004: Data Structures and Algorithms
- **Instructor**: Daniel Wagner
- **Credits**: 4
- **Format**: Hybrid (note that this doesn't match your preferred online format)
- **Level**: Intermediate

#### Course Overview:
This course covers essential data structures such as arrays, linked lists, stacks, queues, trees, graphs, and hash tables. You'll also learn about algorithm complexity and problem-solving techniques, which are foundational skills in computer science and highly relevant to your major.

#### Prerequisites:
You'll need to have completed CS001 as a prerequisite for this course. If you meet this requirement, you will gain practical experience that can enhance your understanding of machine learning and data science c

### 🎯 Understanding the Generated Response

Notice how the LLM's response demonstrates effective context engineering:

**👤 Personalization from User Context:**
- Addresses Sarah by name
- References her intermediate difficulty preference
- Acknowledges her online format preference
- Connects to her interests (machine learning and data science)

**📚 Accuracy from Retrieved Context:**
- Recommends courses from the retrieved list
- Provides correct course details (difficulty, format, credits, description)
- Mentions prerequisites accurately

**🤖 Guidance from System Context:**
- Acts as a supportive advisor
- Explains reasoning for the recommendation
- Stays within the provided course list

This is the power of RAG: the LLM generates a response that is **personalized** (User Context), **accurate** (Retrieved Context), and **helpful** (System Context). Without RAG, the LLM would either hallucinate course details or provide generic advice.

---

## Part 7: Complete RAG Function

Let's combine all three steps into a single, reusable RAG function:

In [36]:
async def hierarchical_rag_query(
    user_query: str,
    user_profile: dict = None,
    summary_limit: int = 5,
    detail_limit: int = 3
):
    """
    Complete RAG pipeline with hierarchical retrieval: Retrieve → Assemble → Generate

    Args:
        user_query: User's question
        user_profile: Optional student profile
        summary_limit: Number of summaries to retrieve
        detail_limit: Number of full details to fetch

    Returns:
        Tuple of (response, summaries, details)
    """
    # Step 1: Retrieve relevant courses (hierarchical)
    summaries, details = await retrieve_courses_hierarchical(
        user_query, summary_limit, detail_limit
    )

    # Step 2: Assemble context
    context = assemble_rag_context(user_query, summaries, details, user_profile)

    # Step 3: Generate response
    response = await generate_response(user_query, context)

    return response, summaries, details


# Test the complete RAG pipeline
print("=" * 60)
print("COMPLETE HIERARCHICAL RAG PIPELINE TEST")
print("=" * 60)
print()

query = "I'm interested in learning about databases and data analytics"
profile = {
    "name": "Alex Johnson",
    "major": "Data Science",
    "year": "Sophomore",
    "interests": ["databases", "data analysis", "SQL"],
    "preferred_difficulty": "intermediate",
    "preferred_format": "hybrid",
}

print(f"Query: {query}")
print()
print(f"Student: {profile['name']} ({profile['major']}, {profile['year']})")
print()

response, summaries, details = await hierarchical_rag_query(query, profile)

print("Retrieved Courses:")
for i, s in enumerate(summaries, 1):
    marker = "★" if i <= len(details) else "○"
    print(f"   {marker} {s.course_code}: {s.title}")
print()

print("AI Response:")
print(response)

COMPLETE HIERARCHICAL RAG PIPELINE TEST

Query: I'm interested in learning about databases and data analytics

Student: Alex Johnson (Data Science, Sophomore)

13:55:24 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I'm interested in learning about databases and data analytics' (summaries=5, details=3)
13:55:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
13:55:25 redis_context_course.hierarchical_manager INFO   Found 5 course summaries for query: I'm interested in learning about databases and data analytics
13:55:25 redis_context_course.hierarchical_manager INFO   Fetched 3 course details
13:55:25 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 5 summaries, 3 details
13:55:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Retrieved Courses:
   ★ CS016: Data Science and Analytics
   ★ CS005: Database Systems
   ★ CS020: Big Data Technologies
   

### 🎯 Why This Complete RAG Function Matters

The `hierarchical_rag_query()` function encapsulates the entire RAG pipeline in a single, reusable interface. This is important because:

**1. Simplicity:** One function call handles retrieval → assembly → generation
- No need to manually orchestrate the three steps
- Clean API for building applications

**2. Consistency:** Every query follows the same pattern
- Ensures all three context types are always included
- Reduces errors from missing context

**3. Efficiency:** Progressive disclosure saves tokens
- Summaries for all matches (lightweight)
- Full details only for top matches (on-demand)
- Optimal token usage

**4. Production-Ready:** This pattern scales to real applications
- In Module 4, we'll add memory (conversation history)
- In Module 5, we'll add tools (course enrollment, prerequisites checking)
- The core RAG pattern remains the same

This is the foundation you'll build on throughout the rest of the workshop.

---

## Part 8: Try Different Queries

Let's test our RAG system with various queries to see how it handles different scenarios:

In [30]:
# Test 1: Beginner looking for programming courses
print("=" * 60)
print("TEST 1: Beginner Programming")
print("=" * 60)
print()

query1 = "I'm new to programming and want to start learning"
profile1 = {
    "name": "Maria Garcia",
    "major": "Undeclared",
    "year": "Freshman",
    "interests": ["programming", "technology"],
    "preferred_difficulty": "beginner",
    "preferred_format": "online",
}

response1, summaries1, details1 = await hierarchical_rag_query(query1, profile1)
print(f"\nQuery: {query1}\n")
print("\nAI Response:\n")
print(response1)

TEST 1: Beginner Programming

18:05:24 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I'm new to programming and want to start learning' (summaries=5, details=3)
18:05:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
18:05:24 redis_context_course.hierarchical_manager INFO   Found 5 course summaries for query: I'm new to programming and want to start learning
18:05:24 redis_context_course.hierarchical_manager INFO   Fetched 3 course details
18:05:24 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 5 summaries, 3 details
18:05:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Query: I'm new to programming and want to start learning


AI Response:

Hi Maria! It's wonderful to hear that you're interested in starting your programming journey! Based on your profile, I have two excellent course recommendations that align with your beginner level and pre

In [31]:
# Test 2: Advanced student looking for specialized courses
print("=" * 60)
print("TEST 2: Advanced Machine Learning")
print("=" * 60)
print()

query2 = "I want advanced courses in machine learning and AI"
profile2 = {
    "name": "David Kim",
    "major": "Computer Science",
    "year": "Senior",
    "interests": ["machine learning", "AI", "research"],
    "preferred_difficulty": "advanced",
    "preferred_format": "in-person",
}

response2, summaries2, details2 = await hierarchical_rag_query(query2, profile2)
print(f"\nQuery: {query2}\n")
print("\nAI Response:\n")
print(response2)

TEST 2: Advanced Machine Learning

18:05:30 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I want advanced courses in machine learning and AI' (summaries=5, details=3)
18:05:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
18:05:30 redis_context_course.hierarchical_manager INFO   Found 5 course summaries for query: I want advanced courses in machine learning and AI
18:05:30 redis_context_course.hierarchical_manager INFO   Fetched 3 course details
18:05:30 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 5 summaries, 3 details
18:05:37 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Query: I want advanced courses in machine learning and AI


AI Response:

Based on your profile and interests, I recommend the following advanced in-person courses in machine learning and AI:

### 1. CS012: Machine Learning Fundamentals
- **Instructor**: Jamie Arnold
- 

In [32]:
# Test 3: Business student looking for relevant courses
print("=" * 60)
print("TEST 3: Business Analytics")
print("=" * 60)
print()

query3 = "What courses can help me with business analytics and decision making?"
profile3 = {
    "name": "Jennifer Lee",
    "major": "Business Administration",
    "year": "Junior",
    "interests": ["analytics", "management", "strategy"],
    "preferred_difficulty": "intermediate",
    "preferred_format": "hybrid",
}

response3, summaries3, details3 = await hierarchical_rag_query(query3, profile3)
print(f"\nQuery: {query3}\n")
print()
print("\nAI Response:\n")
print(response3)

TEST 3: Business Analytics

18:05:37 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What courses can help me with business analytics and decision making?' (summaries=5, details=3)
18:05:37 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
18:05:37 redis_context_course.hierarchical_manager INFO   Found 5 course summaries for query: What courses can help me with business analytics and decision making?
18:05:37 redis_context_course.hierarchical_manager INFO   Fetched 3 course details
18:05:37 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 5 summaries, 3 details
18:05:46 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Query: What courses can help me with business analytics and decision making?



AI Response:

Based on your interests in analytics, management, and strategy, as well as your preference for intermediate-level courses in a hybrid format, I r

### Pattern 1: Flat Retrieval (Simple Approach)

**Characteristics:**
- Retrieve all matching documents
- Include all fields from the data
- Minimal processing or filtering

**Use Cases:**
- Small knowledge bases (<100 documents)
- Simple Q&A where all context is relevant
- Prototyping and initial development

**Trade-offs:**
- ✅ Simple to implement
- ✅ No information loss
- ❌ High token usage
- ❌ May include irrelevant information

Let's see this pattern in action:

**Scenario:** A student asks about machine learning courses.

In [33]:
import json
import tiktoken

def count_tokens(text: str, model: str = "gpt-4o-mini") -> int:
    """Count tokens in text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Pattern 1: Flat retrieval - get many courses, include all fields
flat_summaries, _ = await hierarchical_manager.hierarchical_search(
    query="machine learning course",
    summary_limit=30,
    detail_limit=0
)

flat_context = json.dumps(
    [
        {
            "course_code": s.course_code,
            "title": s.title,
            "description": s.short_description,
            "department": s.department,
            "difficulty_level": s.difficulty_level.value,
            "format": s.format.value,
        }
        for s in flat_summaries
    ],
    indent=2,
)

flat_tokens = count_tokens(flat_context)

print(f"""📊 PATTERN 1: Flat Retrieval
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: {len(flat_summaries)} (broad search)
Tokens: {flat_tokens:,}
Format: Raw JSON with all fields

Sample:
{flat_context[:400]}...
""")

18:06:04 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'machine learning course' (summaries=30, details=0)
18:06:05 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
18:06:05 redis_context_course.hierarchical_manager INFO   Found 30 course summaries for query: machine learning course
18:06:05 redis_context_course.hierarchical_manager INFO   Fetched 0 course details
18:06:05 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 30 summaries, 0 details
📊 PATTERN 1: Flat Retrieval
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: 30 (broad search)
Tokens: 1,844
Format: Raw JSON with all fields

Sample:
[
  {
    "course_code": "CS012",
    "title": "Machine Learning Fundamentals",
    "description": "Introduction to machine learning algorithms and applications.",
    "department": "Computer Science",
    "difficulty_level": "advanced",
    "format": "in_person"
  },
  

### Pattern 2: Hierarchical Retrieval (Optimized Approach)

**Characteristics:**
- Two-tier architecture (summaries + details)
- Progressive disclosure (fetch details on-demand)
- LLM-optimized text format

**Use Cases:**
- Large knowledge bases (1000+ documents)
- Complex queries requiring detailed information
- Production systems with token budgets

**Trade-offs:**
- ✅ Highly token-efficient
- ✅ Better relevance through filtering
- ✅ Scales to large datasets
- ❌ More complex to implement
- ❌ Requires careful tuning

Let's see this pattern:

In [34]:
# Pattern 2: Hierarchical retrieval with progressive disclosure
query = "What machine learning courses are available?"

# Use hierarchical RAG system to get relevant courses
hierarchical_summaries, hierarchical_details = await hierarchical_manager.hierarchical_search(
    query=query,
    summary_limit=3,
    detail_limit=2
)

# Use context assembler for LLM-friendly format
hierarchical_context = context_assembler.assemble_hierarchical_context(
    summaries=hierarchical_summaries,
    details=hierarchical_details,
    query=query
)

hierarchical_tokens = count_tokens(hierarchical_context)

print(f"""📊 PATTERN 2: Hierarchical Retrieval
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: {len(hierarchical_summaries)} summaries + {len(hierarchical_details)} details (filtered by relevance)
Tokens: {hierarchical_tokens:,}
Format: LLM-optimized text with progressive disclosure

Context:
{hierarchical_context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Token Reduction: {flat_tokens - hierarchical_tokens:,} tokens ({((flat_tokens - hierarchical_tokens) / flat_tokens * 100):.1f}% reduction)
Cost Savings: ${((flat_tokens - hierarchical_tokens) / 1_000_000) * 2.50:.4f} per request
""")

18:06:05 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What machine learning courses are available?' (summaries=3, details=2)
18:06:05 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
18:06:05 redis_context_course.hierarchical_manager INFO   Found 3 course summaries for query: What machine learning courses are available?
18:06:05 redis_context_course.hierarchical_manager INFO   Fetched 2 course details
18:06:05 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 3 summaries, 2 details
📊 PATTERN 2: Hierarchical Retrieval
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: 3 summaries + 2 details (filtered by relevance)
Tokens: 2,569
Format: LLM-optimized text with progressive disclosure

Context:
# Course Search Results for: What machine learning courses are available?

## Overview of All Matches

Found 3 relevant courses:


### 1. CS012: Machine Learning Fundament

### Comparing LLM Responses

Let's see how different patterns affect the actual responses:

HumanMessage(content='What machine learning courses are available?', additional_kwargs={}, response_metadata={})

In [35]:
# Test with Pattern 1 (Flat Retrieval)
messages_flat = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

Available Courses:
{flat_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]

response_flat = llm.invoke(messages_flat)

print(f"""📊 RESPONSE WITH PATTERN 1 (Flat Retrieval):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{response_flat.content}
""")

18:06:18 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
📊 RESPONSE WITH PATTERN 1 (Flat Retrieval):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
The available machine learning courses are:

1. **Machine Learning Fundamentals (CS012)**
   - **Description:** Introduction to machine learning algorithms and applications.
   - **Department:** Computer Science
   - **Difficulty Level:** Advanced
   - **Format:** In-person

2. **Linear Algebra for Machine Learning (MATH022)**
   - **Description:** Vectors, matrices, and linear transformations for ML applications.
   - **Department:** Mathematics
   - **Difficulty Level:** Intermediate
   - **Format:** In-person

3. **Deep Learning and Neural Networks (CS013)**
   - **Description:** Advanced neural network architectures and deep learning techniques.
   - **Department:** Computer Science
   - **Difficulty Level:** Advanced
   - **Format:** Online

4. **Natural Languag

In [36]:
# Test with Pattern 2 (Hierarchical Retrieval)
messages_hierarchical = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

{hierarchical_context}

Help students find the best course for their needs."""
    ),
    HumanMessage(content=query),
]

response_hierarchical = llm.invoke(messages_hierarchical)

print(f"""📊 RESPONSE WITH PATTERN 2 (Hierarchical Retrieval):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{response_hierarchical.content}
""")

18:06:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
📊 RESPONSE WITH PATTERN 2 (Hierarchical Retrieval):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
There are three machine learning courses available:

### 1. CS012: Machine Learning Fundamentals
- **Department**: Computer Science
- **Instructor**: Jamie Arnold
- **Credits**: 4 
- **Level**: Advanced
- **Format**: In-person
- **Description**: This course provides an introduction to machine learning algorithms and applications, covering supervised and unsupervised learning, neural networks, and deep learning. Students will learn to implement ML algorithms from scratch and use popular frameworks like scikit-learn and TensorFlow.
- **Prerequisites**: CS002, MATH020
- **Tags**: machine learning, ai, data science, python

### 2. CS013: Deep Learning and Neural Networks
- **Department**: Computer Science
- **Instructor**: Lisa Hensley
- **Credits**: 4 
- **Level

### Pattern Comparison Summary

| Aspect | Pattern 1: Flat | Pattern 2: Hierarchical |
|--------|------------------|-------------------------|
| **Token Usage** | High (~{flat_tokens:,}) | Low (~{hierarchical_tokens:,}) |
| **Relevance** | Broad, may include noise | Focused, filtered by relevance |
| **Complexity** | Simple | Moderate |
| **Best For** | Small datasets, prototypes | Production, large datasets |
| **Cost** | Higher per query | Lower per query |

### Choosing the Right Pattern

**Use Pattern 1 (Flat Retrieval) when:**
- You have a small knowledge base (<100 documents)
- All retrieved documents are likely relevant
- Simplicity is more important than optimization
- You're prototyping or in early development

**Use Pattern 2 (Hierarchical Retrieval) when:**
- You have a large knowledge base (1000+ documents)
- Token efficiency matters (production systems)
- You need to scale to many queries
- Response quality is critical

### Next Steps: Progressive Agents

Want to see these patterns in action? Check out the **Progressive Agents Demo** (Module 3) which shows:
- **Stage 1**: Information overload (no optimization)
- **Stage 2**: Context engineering (optimized retrieval)
- **Stage 3**: Full agent with hierarchical retrieval

Each stage builds on the previous one, demonstrating the evolution from basic to production-ready RAG systems.

**The Engineering Mindset:**
- Context is data that requires engineering discipline
- Raw data ≠ Good context
- Systematic transformation: Extract → Clean → Transform → Optimize → Store
- Quality metrics: Relevance, Completeness, Efficiency, Accuracy

---

## 🎓 Key Takeaways

### What You've Learned

**1. RAG Fundamentals**
- RAG dynamically retrieves relevant information instead of hardcoding knowledge
- Vector embeddings enable semantic search (meaning-based, not keyword-based)
- RAG solves the scalability and token efficiency problems of static context

**2. The Hierarchical RAG Pipeline**
```
User Query → Hierarchical Search → Context Assembly → LLM Generation
```
- **Retrieval:** Find relevant documents using vector similarity (summaries first)
- **Progressive Disclosure:** Fetch full details only for top matches
- **Assembly:** Combine system + user + retrieved context
- **Generation:** LLM creates personalized response with full context

**3. Context Engineering in Practice**
- **System Context:** AI role and instructions (static)
- **User Context:** Student profile and preferences (dynamic, user-specific)
- **Retrieved Context:** Relevant courses from vector search (dynamic, query-specific)
- **Integration:** All three context types work together

**4. Technical Implementation**
- **HierarchicalCourseManager**: Two-tier retrieval (summaries + details)
- **HierarchicalContextAssembler**: Progressive disclosure context assembly
- **Benefits**: Efficient token usage, better relevance, scalable architecture

### Best Practices

**Retrieval:**
- Retrieve only what's needed (top-k results)
- Use progressive disclosure (summaries → details)
- Balance between too few (missing info) and too many (wasting tokens) results
- **💡 Research Insight:** Context Rot research shows that distractors (similar-but-wrong information) have amplified negative impact in long contexts. Precision in retrieval matters more than recall.

**Context Assembly:**
- Structure context clearly (system → user → retrieved)
- Include only relevant metadata
- Keep descriptions concise but informative
- Use progressive disclosure for efficiency

**Generation:**
- Use appropriate temperature (0.7 for creative, 0.0 for factual)
- Provide clear instructions in system context
- Let the LLM explain its reasoning

---

## 🚀 What's Next?

### 📊 Module 3: Data Engineering for Context

Now that you understand RAG fundamentals and why context quality matters, Module 3 teaches you to engineer context with production-level rigor:
- Master data engineering workflows for context preparation
- Understand chunking as a design choice (not a default)
- Build production-ready context pipelines
- Optimize context quality with systematic approaches

### 🧠 Module 4: Memory Systems

In this module, you built a RAG system that retrieves relevant information for each query. But there's a problem: **it doesn't remember previous conversations**.

In Module 4, you'll add memory to your RAG system:
- **Working Memory:** Track conversation history within a session
- **Long-term Memory:** Remember user preferences across sessions
- **Agent Memory Server:** Automatic memory extraction and retrieval

### 🤖 Module 5: Building Agents

After adding memory, you'll transform your RAG system into a full agent:
- **Tool Calling:** Let the AI use functions (search, enroll, check prerequisites)
- **LangGraph State Management:** Orchestrate complex multi-step workflows
- **Agent Reasoning:** Plan and execute multi-step tasks
- **Production Patterns:** Error handling, retries, and monitoring

### The Journey

```
Module 1: Context Engineering Fundamentals
    ↓
Module 2: RAG Fundamentals ← You are here
    ↓
Module 3: Data Engineering for Context ← Next
    ↓
Module 4: Memory Systems
    ↓
Module 5: Building Agents (Complete System)
```

---

## 💪 Practice Exercises

Try these exercises to deepen your understanding:

**Exercise 1: Custom Filters**
- Modify the hierarchical search to filter by specific departments
- Try combining multiple filters (difficulty + format + department)

**Exercise 2: Adjust Retrieval**
- Experiment with different `summary_limit` and `detail_limit` values
- Observe how response quality changes with more/fewer retrieved courses

**Exercise 3: Context Optimization**
- Modify the `assemble_rag_context` function to include more/less detail
- Measure token usage and response quality trade-offs

**Exercise 4: Progressive Disclosure Tuning**
- Try different ratios of summaries to details (5:2, 10:3, 3:1)
- Measure token usage and response quality for each

**Exercise 5: Evaluation**
- Create test queries with expected results
- Measure retrieval accuracy (are the right courses retrieved?)
- Measure generation quality (are responses helpful and accurate?)

---

## 📝 Summary

You've built a complete hierarchical RAG system that:
- ✅ Uses two-tier retrieval (summaries + details)
- ✅ Performs semantic search to find relevant courses
- ✅ Applies progressive disclosure for efficient token usage
- ✅ Assembles context from multiple sources (system + user + retrieved)
- ✅ Generates personalized responses using LLMs
- ✅ Handles different query types and user profiles

This RAG system is the foundation for the advanced topics in Modules 3, 4, and 5. You'll build on this exact code to add data engineering, memory, tools, and full agent capabilities.

**Great work!** You've mastered Retrieved Context and built a production-ready hierarchical RAG pipeline. 🎉

---

## 📚 Additional Resources

### **RAG and Vector Search**
- [Retrieval-Augmented Generation Paper](https://arxiv.org/abs/2005.11401) - Original RAG paper by Facebook AI
- [Redis Vector Similarity Search](https://redis.io/docs/stack/search/reference/vectors/) - Official Redis VSS documentation
- [RedisVL Documentation](https://redisvl.com/) - Redis Vector Library for Python
- [LangChain RAG Tutorial](https://python.langchain.com/docs/tutorials/rag/) - Building RAG applications

### **Embeddings and Semantic Search**
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings) - Understanding text embeddings
- [Sentence Transformers](https://www.sbert.net/) - Open-source embedding models
- [HNSW Algorithm](https://arxiv.org/abs/1603.09320) - Hierarchical Navigable Small World graphs

### **LangChain and Redis Integration**
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction) - Framework overview
- [LangChain Redis Integration](https://python.langchain.com/docs/integrations/vectorstores/redis/) - Using Redis with LangChain
- [Redis Python Client](https://redis-py.readthedocs.io/) - redis-py documentation

### **Advanced RAG Techniques**
- [Advanced RAG Patterns](https://blog.langchain.dev/deconstructing-rag/) - LangChain blog on RAG optimization
- [Advanced Search with RedisVL](https://docs.redisvl.com/en/latest/user_guide/11_advanced_queries.html) - Vector, Hybrid, Text, and Keyword Search
- [RAG Evaluation](https://arxiv.org/abs/2309.15217) - Measuring RAG system performance

---

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

**Redis University - Context Engineering Workshop**


In [ ]:


### TODO Add to Appendix: Reciprocal Rank Fusion (RRF)

**RRF** is a simple but effective way to combine rankings from multiple sources:

```python
RRF_score(doc) = Σ (1 / (k + rank_i))

Where:
- rank_i: Position of document in result list i
- k: Constant (typically 60) to reduce impact of high ranks
```

**Example:**
```
Document "CS101":
- Semantic search rank: 1 → 1/(60+1) = 0.0164
- Keyword search rank: 2 → 1/(60+2) = 0.0161
- Combined RRF score: 0.0325

Document "CS999":
- Semantic search rank: 50 → 1/(60+50) = 0.0091
- Keyword search rank: 1 → 1/(60+1) = 0.0164
- Combined RRF score: 0.0255

Result: CS101 ranks higher (appears in both, good semantic match)
```